In [1]:
from keras.applications import VGG16
rows = 224
cols = 224

model = VGG16(weights = 'imagenet', include_top = False, input_shape = (rows, cols, 3))

for layer in model.layers:
    layer.trainable = False

for (i,layer) in enumerate(model.layers):
    print(str(i) + " " + layer.__class__.__name__, layer.trainable)


Using TensorFlow backend.


0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False


In [2]:
def addlayer(bottom_model, num_classes):
    """creates the head of the model that will bw placed on top of the bottom layers"""
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model

In [3]:

model.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [4]:
model.layers

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

num_classes = 2

FC_Head = addlayer(model, num_classes)
modelnew = Model(inputs=model.input, outputs=FC_Head)
print(modelnew.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = ('C:\\Users\\Ravi Malvia\\Desktop\\mlops-ws\\image_dataset\\train')
validation_data_dir = ('C:\\Users\\Ravi Malvia\\Desktop\\mlops-ws\\image_dataset\\test')


train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20, 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
validation_datagen = ImageDataGenerator(rescale=1./255)

train_batchsize = 15
val_batchsize = 10

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(rows, cols),
                                                    batch_size=train_batchsize,
                                                    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                    target_size=(rows, cols),
                                                    batch_size=val_batchsize,
                                                    class_mode='categorical',
                                                    shuffle=False)

Found 100 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [9]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("face_recog_vgg.h5", monitor="val_loss", mode="min", save_best_only = True, verbose=1)
earlystop = EarlyStopping(monitor= 'val_loss', min_delta = 0, patience = 3, verbose = 1, restore_best_weights = True)
callbacks = [earlystop, checkpoint]

modelnew.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.001),metrics=['accuracy'])

nb_train_samples=100
nb_validation_samples=40
epochs=4
batch_size=10

history = modelnew.fit_generator(train_generator,
                                 steps_per_epoch=nb_train_samples // batch_size,
                                 epochs=epochs,
                                 callbacks=callbacks,
                                 validation_data=validation_generator,
                                 validation_steps=nb_validation_samples // batch_size)
modelnew.save("face_recog_vgg.h5")

Epoch 1/4
10/10 [==============================] - 60s 6s/step - loss: 1.4529 - accuracy: 0.4690 - val_loss: 0.3443 - val_accuracy: 1.0000

Epoch 00001: val_loss improved from inf to 0.34435, saving model to face_recog_vgg.h5
Epoch 2/4
10/10 [==============================] - 61s 6s/step - loss: 0.3148 - accuracy: 0.8571 - val_loss: 0.0230 - val_accuracy: 1.0000

Epoch 00002: val_loss improved from 0.34435 to 0.02305, saving model to face_recog_vgg.h5
Epoch 3/4
10/10 [==============================] - 64s 6s/step - loss: 0.0121 - accuracy: 1.0000 - val_loss: 0.0012 - val_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.02305 to 0.00124, saving model to face_recog_vgg.h5
Epoch 4/4
10/10 [==============================] - 63s 6s/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 2.3837e-04 - val_accuracy: 1.0000

Epoch 00004: val_loss improved from 0.00124 to 0.00024, saving model to face_recog_vgg.h5


In [10]:
from keras.models import load_model
classifier = load_model('face_recog_vgg.h5')

In [28]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

actors_dataset_dict = {"[0]": "Ravi", 
                      "[1]": "Varsha",
                      
                      }



def draw_test(name, pred, im):
    actors = actors_dataset_dict[str(pred)]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, actors, (0, 40) , cv2.FONT_HERSHEY_SIMPLEX,1, (0,0,255), 2)
    cv2.imshow(name, expanded_image)
    
def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + str(path_class))
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)    


input_im = getRandomImage("C:\\Users\\Ravi Malvia\\Desktop\\mlops-ws\\image_dataset\\test\\")
input_original = input_im.copy()
input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)
    
input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
input_im = input_im / 255.
input_im = input_im.reshape(1,224,224,3) 
    
    
 # Get Prediction

res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)
    
    # Show image with predicted class
draw_test("Prediction", res, input_original) 
cv2.waitKey(5000)
cv2.destroyAllWindows()

Class - varsha
